In [ ]:
import psycopg2

In [ ]:
import psycopg2.extensions

In [ ]:
dsn = "postgresql://michael:@/landuse?host=/run/postgresql"

In [ ]:
conn = psycopg2.connect(**psycopg2.extensions.parse_dsn(dsn))

In [ ]:
def send_sql(connection, query, **kwargs):
    import time
    args = kwargs.get("args")
    quiet = kwargs.get("quiet", False)
    return_result = kwargs.get("return_result", False)
    start = time.time()
    cursor = connection.cursor()
    result = None
    rowcount = -1
    try:
        cursor.execute(query, args)
        rowcount = cursor.rowcount
        if return_result:
            result = cursor.fetchall()
        connection.commit()
    except Exception as ex:
        print("Failed to execute SQL statement:\n{}\nReason: {}".format(cursor.query, ex))
        connection.rollback()
    finally:
        cursor.close()
        msg = "Executed SQL command in {:0.3f} seconds.".format(time.time() - start)
        if rowcount >= 0:
            msg = "{}, affected {} rows.".format(msg[:-1], rowcount)
        print(msg)
    return result

## Prepare database

We have to make the area_id column of the table `land` unique and calculate the area of each polygon.

In [ ]:
send_sql(conn, "ALTER TABLE land ADD CONSTRAINT area_id_unique UNIQUE(area_id);")

## Municipalities as polygons

First, we have to get the polygon of Germany

In [ ]:
sql = "SELECT geom FROM admin_boundaries WHERE admin_level = 4 AND name = 'Baden-Württemberg';"
germany_wkb = send_sql(conn, sql, return_result=True)[0]
#c.execute(sql)
#germany_wkb = c.fetchone()[0]

Create a database table of municipalities

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS cities;")
send_sql(conn, """CREATE TABLE cities AS
SELECT
    area_id, admin_level, name, tags, geom
  FROM admin_boundaries
  WHERE admin_level = 8 AND ST_Intersects(geom, %s) AND ST_CoveredBy(geom, %s);
""", args=(germany_wkb, germany_wkb))
send_sql(conn, "CREATE INDEX ON cities USING gist(geom);")
send_sql(conn, "ALTER TABLE cities ADD CONSTRAINT cities_area_id_unique UNIQUE(area_id);")

Add "kreisfreie Städte" (AL 6 polygons with no AL 8 inside them):

In [ ]:
sql = """INSERT INTO cities (area_id, admin_level, name, tags, geom)
  SELECT
      area_id,
      admin_level,
      name,
      tags,
      geom
    FROM admin_boundaries AS a
    WHERE
      admin_level = %s
      AND (SELECT c.geom FROM cities AS c WHERE ST_Contains(a.geom, c.geom) LIMIT 1) IS NULL;"""
for level in [6, 4]:
    send_sql(conn, sql, args=(level,))

Create index on name and area_id for later use.

In [ ]:
send_sql(conn, "CREATE INDEX IF NOT EXISTS cities_area_id_name_idx ON cities USING btree(area_id, name);")

## A dataset of non-overlapping landuse

In OpenStreetMap, landuse polygons often overlap each other – often in the case when one landuse is a "hole" within another polygon (e.g. lake inside a forest). Before we can reliably determine a matrix of neighbouring landuse classes, we have to get rid of all overlaps.

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS land_without_overlaps;")
sql = """CREATE TABLE land_without_overlaps AS
SELECT
    l.area_id AS area_id,
    l.feature AS feature,
    l.tags AS tags,
    ST_CollectionExtract(
      ST_Difference(
        l.geom,
        ST_Union(
          COALESCE(
            o.geom,
            ST_SetSRID(
              ST_GeometryFromText('POLYGON EMPTY'),
              4326)
            )
          )
        ),
      3) AS geom
  FROM land AS l
  LEFT OUTER JOIN land AS o
    ON o.area < l.area AND ST_Intersects(l.geom, o.geom) AND l.geom && o.geom AND l.area_id <> o.area_id
  GROUP BY l.area_id, l.feature, l.tags, l.geom;
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX land_without_overlaps_geom_idx ON land_without_overlaps USING gist(geom);")

## Landuse polygons clipped to municipalities

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_by_municipality;")
sql = """CREATE TABLE landuse_by_municipality AS
SELECT 
    l.area_id AS landuse_id,
    c.area_id AS city_id,
    l.tags AS tags,
    CASE
      WHEN ST_Within(l.geom, c.geom) THEN l.geom
      ELSE ST_CollectionExtract(ST_Intersection(l.geom, c.geom), 3)
    END AS geom
  FROM land_without_overlaps AS l
  JOIN cities AS c
    ON l.geom && c.geom AND ST_Intersects(l.geom, c.geom) AND ST_Relate(l.geom, c.geom, 'T********');
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_by_municipality_city_id ON landuse_by_municipality USING BTREE(city_id);")

## Landuse coverage per municipality

Overlaps were eliminated in a previous step.

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_coverage_per_municipality;")
sql = """CREATE TABLE landuse_coverage_per_municipality AS
SELECT
    l.city_id AS city_id,
    ST_Area(c.geom::geography) AS city_area,
    SUM(ST_Area(l.geom::geography)) AS area_size,
    c.geom AS city_geom,
    c.name AS city_name
  FROM cities AS c
  LEFT OUTER JOIN landuse_by_municipality AS l ON c.area_id = l.city_id
  GROUP BY city_id, c.name, c.geom;
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_coverage_per_municipality_id ON landuse_coverage_per_municipality USING BTREE(city_id);")
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_coverage_per_municipality_geom ON landuse_coverage_per_municipality USING GIST(city_geom);")

## Overlaps of landuse

Find intersections between landuse polygons:

In [ ]:
send_sql(conn, "CREATE INDEX IF NOT EXISTS land_area_size_idx ON land USING btree(area);")
send_sql(conn, "DROP TABLE IF EXISTS landuse_overlaps;")
sql = """CREATE TABLE landuse_overlaps AS
SELECT
    ST_CollectionExtract(ST_Intersection(a.geom, b.geom), 3) AS geom,
    a.area_id AS id1,
    b.area_id AS id2,
    a.feature AS feature1,
    b.feature AS feature2,
    a.area AS size_1,
    b.area AS size_2
  FROM land AS a
  JOIN land AS b
    ON
    a.area >= b.area
    AND a.geom && b.geom
    AND ST_Intersects(a.geom, b.geom)
    AND ST_Relate(a.geom, b.geom, 'T********')
    AND a.area_id != b.area_id;
"""
send_sql(conn, sql)

Create a matrix of feature classes overlapping each other by area. `feature1` is the larger polygon, `feature2` the smaller one.

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_overlaps_feature_classes;")
sql = """CREATE TABLE landuse_overlaps_feature_classes AS
SELECT
    feature1,
    feature2,
    SUM(ST_Area(geom::geography)) AS area_size,
    COUNT(1) AS occurences
  FROM landuse_overlaps
  GROUP BY feature1, feature2;
"""
send_sql(conn, sql)

Drop distinction between larger and smaller polygons, just make a matrix of landuse classes:

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_overlaps_feature_classes_ordered;")
sql = """CREATE TABLE landuse_overlaps_feature_classes_ordered AS
SELECT
    f1 AS feature1,
    f2 AS feature2,
    SUM(area_size) AS area_size,
    COUNT(1) AS occurences
  FROM (
    SELECT
        CASE WHEN feature1 < feature2 THEN feature1 ELSE feature2 END AS f1,
        CASE WHEN feature1 < feature2 THEN feature2 ELSE feature1 END AS f2,
        ST_Area(geom::geography) AS area_size
      FROM landuse_overlaps
  ) AS a
  GROUP BY f1, f2;
"""
send_sql(conn, sql)

## Generic stats about landuses (sizes, node count, boundary length)

Get boundary lines of landuse polygons (as (multi)linestrings) and the area size.

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_boundary_lines;")
sql = """CREATE TABLE landuse_boundary_lines AS
SELECT
    ST_Boundary(geom) AS geom,
    ST_Length(ST_Boundary(geom)::geography) AS length,
    ST_NPoints(geom) AS node_count,
    ST_Area(geom::geography) AS area_size,
    area_id,
    feature,
    tags
  FROM land;
"""
send_sql(conn, sql)
#send_sql(conn, "CREATE INDEX landuse_boundary_lines_length_idx ON landuse_boundary_lines USING btree(length);")
#send_sql(conn, "CREATE INDEX landuse_boundary_lines_node_count_idx ON landuse_boundary_lines USING btree(node_count);")
#send_sql(conn, "CREATE INDEX landuse_boundary_lines_area_size_idx ON landuse_boundary_lines USING btree(area_size);")

Find polygons with long boundary but small area size:

In [ ]:
sql = '''SELECT
      area_id,
      feature,
      (sqrt(area_size)/length) AS size_length_ratio,
      area_size
    FROM landuse_boundary_lines
  ORDER BY size_length_ratio ASC NULLS LAST;
'''

Find polygons with many nodes per boundary length:

In [ ]:
sql = '''SELECT
        area_id,
        feature,
        area_size,
        (length/node_count::REAL) AS length_by_nodes,
        length,
        node_count
      FROM landuse_boundary_lines
      ORDER BY length_by_nodes ASC NULLS LAST;
'''

Unfortunately, the result is a bit worthless. The lowest ratio of length and node count is found at small circular areas (less than 1 m²). The largest ratio is found at rectangular forests (or rectangular grass areas on airports). This is not interesting.

In [ ]:
sql = '''SELECT
        area_id,
        feature,
        area_size,
        (node_count::REAL/area) AS nodes_per_area,
        length,
        node_count
      FROM landuse_boundary_lines
      ORDER BY length_by_nodes ASC NULLS LAST;
'''

The ratio of node count and area size does not return better results. The top and bottom entries of the list are almost identical.

Municipalities with many/few nodes per area:

In [ ]:
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_boundary_lines_geom_idx ON landuse_boundary_lines USING gist(geom);")
send_sql(conn, "DROP TABLE IF EXISTS nodes_per_municipality;")
sql = '''CREATE TABLE nodes_per_municipality AS
SELECT
    name,
    area_id,
    node_count/(ST_Area(area::geography) / 10000) AS nodes_per_area
  FROM (
    SELECT
        c.name,
        c.area_id,
        SUM(ST_NPoints(ST_Intersection(c.geom, b.geom))) AS node_count,
        c.geom AS area
      FROM cities AS c
      JOIN landuse_boundary_lines AS b
        ON c.geom && b.geom AND ST_Intersects(c.geom, b.geom)
      GROUP BY c.area_id, c.name, area
  ) AS a
  ORDER BY node_count DESC NULLS LAST;
'''
send_sql(conn, sql)

**TODO**

* determine node count per area and per boundary length, its distribution and differences between municipalities
* clip landuse boundaries to municipal polygons
* determine ratio of sqrt(area size) to boundary length, its distribution and differences between municipalities

## Multipolygons vs. simple polygons

Find multipolygons and compute number of inner and outer rings per multipolygon:

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS multipolygon_landuse;")
sql = """CREATE TABLE multipolygon_landuse AS
SELECT
    geom,
    area_id,
    COUNT(1) AS outer_rings,
    SUM(ST_NumInteriorRing(dumped_geom)) AS inner_rings,
    mp_outer_way_count,
    feature,
    tags
  FROM (
    SELECT
        geom,
        area_id,
        (ST_Dump(geom)).geom AS dumped_geom,
        mp_outer_way_count,
        feature,
        tags
      FROM land
      WHERE area_id < 0
    ) AS a
  GROUP BY area_id, mp_outer_way_count, feature, tags, geom;
"""
send_sql(conn, sql)

Unnecessary multipolygon per municipality:

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS unnecessary_multipolygons_per_municipality;")
sql = """CREATE TABLE unnecessary_multipolygons_per_municipality AS
SELECT
    ST_CollectionExtract(ST_Intersection(c.geom, m.geom), 3) AS geom,
    m.feature AS feature,
    m.outer_rings AS outer_rings,
    m.inner_rings AS inner_rings,
    m.mp_outer_way_count AS outer_way_count,
    m.tags AS tags,
    c.area_id AS city_id,
    m.area_id AS multipolygon_id
  FROM multipolygon_landuse AS m
  JOIN cities AS c
    ON
      c.geom && m.geom
      AND ST_Intersects(c.geom, m.geom)
      AND ST_Relate(c.geom, m.geom, 'T********')
      AND (
        (m.outer_rings > 1 AND NOT m.tags?'name')
        OR (m.mp_outer_way_count > 1 AND m.outer_rings = 1)
      );
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX IF NOT EXISTS unnecessary_multipolygons_per_municipality_city_id ON unnecessary_multipolygons_per_municipality USING BTREE(city_id);")

Create table to store base values for the multipolygonitis score calculation:

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS multipolygonitis_score;")
sql = """CREATE TABLE multipolygonitis_score AS
SELECT
    SUM(ST_Area(l.geom::geography)) AS landuse_area,
    COUNT(l.geom) AS landuse_count,
    a.multipolygon_area AS multipolygon_area,
    a.multipolygon_count AS multipolygon_count,
    a.city_id AS city_id,
    a.city_name AS city_name,
    a.city_area AS city_area,
    a.city_geom AS city_geom
  FROM (
    SELECT
      SUM(ST_Area(m.geom::geography)) AS multipolygon_area,
      COUNT(m.geom) AS multipolygon_count,
      c.area_id AS city_id,
      c.name AS city_name,
      ST_Area(c.geom::geography) AS city_area,
      c.geom AS city_geom
    FROM cities AS c
    LEFT OUTER JOIN unnecessary_multipolygons_per_municipality AS m
      ON m.city_id = c.area_id
    GROUP BY c.area_id, c.name, c.geom
  ) AS a
  LEFT OUTER JOIN landuse_by_municipality AS l
    ON l.city_id = a.city_id
  GROUP BY a.city_id, a.city_name, a.multipolygon_area, a.city_area, a.multipolygon_count, a.city_geom;
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX IF NOT EXISTS multipolygonitis_score_geom_idx ON multipolygonitis_score USING gist(city_geom);")

## Road segments which are used by landuse polygons

Get boundary lines of all landuse polygons:

In [ ]:
sql = '''
DROP TABLE IF EXISTS landuse_boundaries;
CREATE TABLE landuse_boundaries AS
  SELECT
      ST_CollectionExtract(ST_Boundary(geom), 2) AS geom,
      area_id,
      feature,
      tags
    FROM land;
'''
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX landuse_boundaries_geom_idx ON landuse_boundaries USING gist(geom);")
send_sql(conn, "CREATE INDEX landuse_boundaries_id_idx ON landuse_boundaries USING btree(area_id);")

Get intersection of these boundary lines with other landuse boundary lines:

In [ ]:
sql = '''
DROP TABLE IF EXISTS landuse_landuse_boundaries;
CREATE TABLE landuse_landuse_boundaries AS
SELECT
    ST_CollectionExtract(ST_Intersection(l1.geom, l2.geom), 2) AS geom,
    l1.feature AS feature1,
    l2.feature AS feature2,
    l1.area_id AS land_id1,
    l2.area_id AS land_id2,
    l1.tags AS tags1,
    l2.tags As tags2
  FROM landuse_boundaries AS l1
  JOIN landuse_boundaries AS l2
    -- Ensures that we do not get any duplicates.
    -- Using < instead of <> because this makes use of the index.
    ON l1.area_id < l2.area_id
    AND l1.geom && l2.geom
    AND ST_Intersects(l1.geom, l2.geom)
    AND ST_Relate(l1.geom, l2.geom, '1********');
'''
send_sql(conn, sql)

Find landuse polygons sharing their boundary with roads.

In [ ]:
sql = '''
DROP TABLE IF EXISTS landuse_boundaries_on_roads;
SELECT
    ST_CollectionExtract(ST_Intersection(ST_Boundary(l.geom), r.geom), 2) AS geom,
    l.feature AS land_feature,
    l.area_id AS land_id,
    COALESCE(r.highway, r.railway) AS road_feature,
    r.way_id AS road_id
  INTO landuse_boundaries_on_roads
  FROM land AS l
  JOIN streets AS r ON l.geom && r.geom AND ST_Intersects(l.geom, r.geom) AND ST_Relate(l.geom, r.geom, '***1*****');
'''
send_sql(conn, sql)

The table `landuse_boundaries_on_roads` contains duplicates if a road segment belongs to two landuse polygons. They are de-duplicated now:

In [ ]:
sql = '''
DROP TABLE IF EXISTS roads_as_landuse_boundaries;
CREATE INDEX IF NOT EXISTS landuse_boundaries_on_roads_road_id_feature ON landuse_boundaries_on_roads USING btree(road_id, road_feature);
CREATE TABLE roads_as_landuse_boundaries AS
SELECT
    ST_CollectionExtract(ST_Union(ST_ClusterIntersecting(geom)), 2) AS geom,
    road_feature,
    road_id
  FROM landuse_boundaries_on_roads
  GROUP BY road_id, road_feature;
'''
send_sql(conn, sql)

Get a geometry collection of the roads of each municipality.

Performance notes:

* In the `SELECT` part, `ST_Within` is faster than using `ST_CoveredBy` or `ST_Relate` (saves 25%).
* In the `JOIN` condition, not using `ST_Relate(rl.geom, c.geom, '1********')` saves additional 90% of time.

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS cities_road_network_length;")
sql = '''CREATE TABLE cities_road_network_length
  AS SELECT
    COALESCE(ST_Length(
      ST_Collect(ST_CollectionExtract(
        CASE
          WHEN ST_Within(rl.geom, c.geom) THEN rl.geom
          ELSE ST_Intersection(rl.geom, c.geom)
        END,
      2))::geography
    ), 0::REAL) AS road_length,
    c.area_id AS city_id,
    c.name AS city_name,
    c.geom AS city_geom
  FROM cities AS c
  LEFT OUTER JOIN streets AS rl
    ON c.geom && rl.geom AND ST_Intersects(c.geom, rl.geom)
  GROUP BY c.area_id, c.name, c.geom;
'''
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX cities_road_network_length_area_id_idx ON cities_road_network_length USING btree(city_id);")
send_sql(conn, "CREATE INDEX cities_road_network_length_geom_idx ON cities_road_network_length USING gist(city_geom);")

Intersect length of this road network per municpality and get a GeometryCollection of all roads per municipality:

In [ ]:
send_sql(conn, "CREATE INDEX IF NOT EXISTS roads_as_landuse_boundaries_geom_idx ON roads_as_landuse_boundaries USING GIST(geom);")
send_sql(conn, "DROP TABLE IF EXISTS roads_as_landuse_boundaries_per_city;")
sql = """CREATE TABLE roads_as_landuse_boundaries_per_city AS
SELECT
      COALESCE(ST_Collect(
        ST_CollectionExtract(
          CASE
            WHEN ST_Within(rl.geom, c.geom) THEN rl.geom
            ELSE ST_Intersection(rl.geom, c.geom)
          END,
          2)
      ), ST_SetSRID(ST_GeometryFromText('LINESTRING EMPTY'), 4326)) AS roads_geom,
      c.area_id AS city_id,
      c.name AS city_name,
      c.geom AS city_geom
    FROM cities AS c
    LEFT OUTER JOIN roads_as_landuse_boundaries AS rl
      ON
        c.geom && rl.geom
        AND ST_Intersects(c.geom, rl.geom)
    GROUP BY c.area_id, c.name, c.geom;
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX IF NOT EXISTS roads_as_landuse_boundaries_per_city_idx ON roads_as_landuse_boundaries_per_city USING btree(city_id);")

Do the same for landuse boundaries on roads (if a road is used by two neighbouring landuse polygons, it will appear twice):

In [ ]:
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_boundaries_on_roads_geom_idx ON landuse_boundaries_on_roads USING GIST(geom);")
send_sql(conn, "DROP TABLE IF EXISTS landuse_boundaries_on_roads_per_city;")
sql = """CREATE TABLE landuse_boundaries_on_roads_per_city AS
SELECT
      COALESCE(ST_Collect(
        ST_CollectionExtract(
          CASE
            WHEN ST_Within(rl.geom, c.geom) THEN rl.geom
            ELSE ST_Intersection(rl.geom, c.geom)
          END,
          2)
      ), ST_SetSRID(ST_GeometryFromText('LINESTRING EMPTY'), 4326)) AS roads_geom,
      c.area_id AS city_id,
      c.name AS city_name,
      c.geom AS city_geom
    FROM cities AS c
    LEFT OUTER JOIN landuse_boundaries_on_roads AS rl
      ON
        c.geom && rl.geom
        AND ST_Intersects(c.geom, rl.geom)
    GROUP BY c.area_id, c.name, c.geom;
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_boundaries_on_roads_per_city_idx ON landuse_boundaries_on_roads_per_city USING btree(city_id);")

Determine per municipality how large the proportion of landuse boundaries glued to roads compared to all roads is.

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS roads_landuse_boundary_fraction_per_city;")
sql = """CREATE TABLE roads_landuse_boundary_fraction_per_city AS
SELECT
    landuse_boundaries_on_roads_length / landuse_boundary_length AS landuse_boundaries_on_roads_fraction,
    landuse_boundaries_on_roads_length AS landuse_boundaries_on_roads,
    landuse_boundary_length,
    roads_as_landuse_boundary_length / road_network_length AS roads_as_landuse_boundary_fraction,
    roads_as_landuse_boundary_length,
    road_network_length,
    coverage_fraction,
    city_id,
    city_name,
    city_geom
FROM (
  SELECT
    ST_Length(br.roads_geom::geography) AS landuse_boundaries_on_roads_length,
    ST_Length(b.geom::geography) AS landuse_boundary_length,
    ST_Length(rb.roads_geom::geography) AS roads_as_landuse_boundary_length,
    c.road_length AS road_network_length,
    cov.area_size/cov.city_area AS coverage_fraction,
    c.city_name AS city_name,
    c.city_id AS city_id,
    c.city_geom AS city_geom
  FROM cities_road_network_length AS c
  LEFT OUTER JOIN roads_as_landuse_boundaries_per_city AS rb
    ON rb.city_id = c.city_id
  LEFT OUTER JOIN landuse_boundaries_per_city AS b
    ON b.city_id = c.city_id
  LEFT OUTER JOIN landuse_boundaries_on_roads_per_city AS br
    ON br.city_id = c.city_id
  LEFT OUTER JOIN landuse_coverage_per_municipality AS cov
    ON cov.city_id = c.city_id
  ) AS a;
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX IF NOT EXISTS roads_landuse_boundary_fraction_per_city_geom_idx ON roads_landuse_boundary_fraction_per_city USING gist(city_geom);")

Determine per municipality how large the proportion of landuse boundaries glued to other landuse boundaries compared to all landuse boundaries is.

In [ ]:
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_landuse_boundaries_geom_idx ON landuse_landuse_boundaries USING gist(geom);")
sql = """
DROP TABLE IF EXISTS landuse_boundaries_per_city;
CREATE TABLE landuse_boundaries_per_city AS
SELECT
    ST_CollectionExtract(ST_Collect(
      CASE
        WHEN ST_Within(lb.geom, c.geom) THEN lb.geom
        ELSE ST_Intersection(lb.geom, c.geom)
      END
    ), 2) AS geom,
    c.area_id AS city_id
  FROM cities AS c
  LEFT OUTER JOIN landuse_boundaries AS lb
    ON
      c.geom && lb.geom
      AND ST_Intersects(c.geom, lb.geom)
  GROUP BY c.area_id;
"""
send_sql(conn, sql)
sql = """
DROP TABLE IF EXISTS landuse_landuse_boundaries_per_city;
CREATE TABLE landuse_landuse_boundaries_per_city AS
SELECT
    ST_CollectionExtract(ST_Collect(
      CASE
        WHEN ST_Within(llb.geom, c.geom) THEN llb.geom
        ELSE ST_Intersection(llb.geom, c.geom)
      END
    ), 2) AS geom,
    c.area_id AS city_id
  FROM cities AS c
  LEFT OUTER JOIN landuse_landuse_boundaries AS llb
    ON
      c.geom && llb.geom
      AND ST_Intersects(c.geom, llb.geom)
  GROUP BY c.area_id;
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_boundaries_per_city_city_id ON landuse_boundaries_per_city USING btree(city_id);")
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_landuse_boundaries_per_city_city_id ON landuse_landuse_boundaries_per_city USING btree(city_id);")
sql = """DROP TABLE IF EXISTS landuse_landuse_boundary_fraction_per_municipality;
CREATE TABLE landuse_landuse_boundary_fraction_per_municipality AS
SELECT
    bb_len/b_len AS bounds_fraction,
    bb_len AS boundary_boundary_length,
    b_len boundary_length,
    city_id,
    name AS city_name,
    city_geom,
    coverage_fraction
FROM (
  SELECT
    -- Multiplication by 2 because boundary-boundary intersections were deduplicated in a previous step.
    ST_Length(bb.geom::geography) * 2 AS bb_len,
    ST_Length(b.geom::geography) AS b_len,
    cov.area_size/cov.city_area AS coverage_fraction,
    c.name AS name,
    c.area_id AS city_id,
    c.geom AS city_geom
  FROM cities AS c
  LEFT OUTER JOIN landuse_landuse_boundaries_per_city AS bb
    ON bb.city_id = c.area_id
  LEFT OUTER JOIN landuse_boundaries_per_city AS b
    ON b.city_id = c.area_id
  LEFT OUTER JOIN landuse_coverage_per_municipality AS cov
    ON cov.city_id = c.area_id
  ) AS a;
"""
send_sql(conn, sql)

When inspecting the `landuse_landuse_boundary_fraction_per_municipality` table, you should keep in mind that municipalities without almost complete landuse coverage will have high fractions of landuse boundaries not sharing nodes with other landuse boundaries because they have no neighbours.

## Landuse polygons almost touching each other or roads

As a first step, we find all landuse boundary line which are not shared with other landuse polygons.
Shorten the linestrings and caluclate buffers.

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS lonely_landuse_boundaries;")
buffer_dist = 1.0
sql = """CREATE TABLE lonely_landuse_boundaries AS
SELECT
    ST_Transform(ST_LineSubstring(geom_3857, cutoff_fraction, 1.0-cutoff_fraction), 4326)::geography AS geog_for_buffer,
    geom AS geom,
    geom::geography AS geog,
    feature,
    mercator_factor,
    area_id
  FROM (
    SELECT
        -- buffer_dist multiplied by Mercator factor because ST_LineSubString works with Geometry type.
        ({} * mercator_factor) / length AS cutoff_fraction,
        mercator_factor,
        ST_Transform(geom, 3857) AS geom_3857,
        geom,
        feature,
        area_id
      FROM (
        SELECT
            1.0 / cos(RADIANS(ST_Y(ST_StartPoint(geom)))) AS mercator_factor,
            ST_Length(geom::geography) AS length,
            geom,
            feature,
            area_id
          FROM (
            SELECT
                (ST_Dump(
                  ST_CollectionExtract(
                    ST_Difference(
                      c.geom,
                      ST_Collect(COALESCE(rlb.geom, ST_SetSRID(ST_GeomFromText('LINESTRING EMPTY'), 4326)))
                    ),
                  2)
                )).geom AS geom,
                c.feature AS feature,
                c.area_id AS area_id
              FROM (
                SELECT
                    ST_Difference(
                      lb.geom, ST_Collect(COALESCE(llb.geom, ST_SetSRID(ST_GeomFromText('LINESTRING EMPTY'), 4326)))
                    ) AS geom,
                    lb.feature AS feature,
                    lb.area_id AS area_id
                  FROM landuse_boundaries AS lb
                  LEFT OUTER JOIN landuse_landuse_boundaries AS llb
                    ON lb.geom && llb.geom AND ST_Intersects(lb.geom, llb.geom)
                  GROUP BY lb.area_id, lb.feature, lb.geom
              ) AS c
              LEFT OUTER JOIN roads_as_landuse_boundaries AS rlb
                ON c.geom && rlb.geom AND ST_Intersects(c.geom, rlb.geom)
              GROUP BY c.area_id, c.feature, c.geom
          ) AS d
      ) AS b
      WHERE length > 5 * {}
  ) AS a;
""".format(buffer_dist * 2.0, buffer_dist * 2.0)
send_sql(conn, sql)

Find intersections:

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS closeby_landuse_intersections;")
send_sql(conn, "CREATE INDEX IF NOT EXISTS lonely_landuse_boundaries_buffer ON lonely_landuse_boundaries USING gist(geog_for_buffer);")
send_sql(conn, "ALTER TABLE streets ADD COLUMN IF NOT EXISTS geog geography; UPDATE streets SET geog = geom::geography WHERE geog IS NULL;")
sql = """CREATE INDEX IF NOT EXISTS streets_geog ON streets USING gist(geog)
  WHERE highway IN ('motorway', 'motorway_link', 'trunk', 'trunk_link', 'primary', 'primary_link',
      'secondary', 'secondary_link', 'tertiary', 'tertiary_link', 'unclassified', 'residential',
      'living_street', 'pedestrian', 'service', 'track', 'busway', 'bus_guideway')
    OR railway IN ('rail', 'narrow_gauge', 'tram', 'subway', 'ligh_rail', 'monorail', 'funicular');
"""
send_sql(conn, sql)
sql = """CREATE TABLE closeby_landuse_intersections AS
SELECT
    ST_CollectionExtract(ST_Intersection(ST_Buffer(a.geog_for_buffer, {buffer_dist})::geometry, b.geom), 2) AS geom,
    a.area_id AS id1,
    b.area_id AS id2,
    a.feature AS feature1,
    b.feature AS feature2,
    TRUE AS touches_to_landuse
  FROM lonely_landuse_boundaries AS a
  JOIN lonely_landuse_boundaries AS b
    ON ST_DWithin(a.geog_for_buffer, b.geog, {buffer_dist}) AND a.area_id < b.area_id
UNION ALL
SELECT
    ST_CollectionExtract(ST_Intersection(ST_Buffer(a.geog_for_buffer, {buffer_dist})::geometry, r.geom), 2) AS geom,
    a.area_id AS id1,
    r.way_id AS id2,
    a.feature AS feature1,
    COALESCE(r.highway, r.railway) AS feature2,
    FALSE AS touches_to_landuse
  FROM lonely_landuse_boundaries AS a
  JOIN streets AS r
    ON ST_DWithin(a.geog_for_buffer, r.geog, {buffer_dist})
      AND (
        r.highway IN ('motorway', 'motorway_link', 'trunk', 'trunk_link', 'primary', 'primary_link',
          'secondary', 'secondary_link', 'tertiary', 'tertiary_link', 'unclassified', 'residential',
          'living_street', 'pedestrian', 'service', 'track', 'busway', 'bus_guideway')
        OR r.railway IN ('rail', 'narrow_gauge', 'tram', 'subway', 'ligh_rail', 'monorail', 'funicular')
      );
""".format(buffer_dist=buffer_dist)
send_sql(conn, sql)

Table of length of boundaries with near-by touches per municipality:

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS closeby_landuse_intersections_per_municipality;")
send_sql(conn, "CREATE INDEX IF NOT EXISTS closeby_landuse_intersections_geom ON closeby_landuse_intersections USING gist(geom);")
send_sql(conn, "CREATE INDEX IF NOT EXISTS landuse_landuse_boundary_fraction_per_municipality_city_id ON landuse_landuse_boundary_fraction_per_municipality USING btree(city_id);")
sql = """CREATE TABLE closeby_landuse_intersections_per_municipality AS
SELECT
    ST_Length(ST_CollectionExtract(ST_Collect(ST_Intersection(i.geom, c.geom)), 2)::geography) AS length,
    i.touches_to_landuse AS touches_to_landuse,
    llb.boundary_length AS boundary_length,
    c.area_id AS city_id,
    c.name AS name
  FROM cities AS c
  JOIN closeby_landuse_intersections AS i
    ON c.geom && i.geom AND ST_Intersects(c.geom, i.geom)
  JOIN landuse_landuse_boundary_fraction_per_municipality AS llb
    ON c.area_id = llb.city_id
  GROUP BY c.area_id, c.name, i.touches_to_landuse, llb.boundary_length, c.geom;
"""
send_sql(conn, sql)

## Age of landuse polygons

**TODO**

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_nodes;")
send_sql(conn, "CREATE INDEX IF NOT EXISTS land_rel_id_idx ON land USING btree(area_id) WHERE area_id < 0;")

# very slow (I stopped the query after more than 5 minutes)
#sql = """CREATE TABLE landuse_relation_nodes AS
#SELECT
#    relways.landuse_id AS landuse_id,
#    relways.feature AS feature,
#    n.id AS node_id,
#    ST_SetSRID(ST_MakePoint(n.lon, n.lat), 4326) AS node_geom,
#    n.created AS last_modified,
#    n.version AS version
#  FROM planet_osm_nodes AS n
#  JOIN (
#    SELECT
#        l.area_id AS landuse_id,
#        l.feature AS feature,
#        w.id AS way_id,
#        w.nodes AS nodes
#      FROM land AS l
#      JOIN planet_osm_rels AS r ON -l.area_id = r.id AND l.area_id < 0
#      JOIN planet_osm_ways AS w ON r.members @> ('[{"ref": ' || w.id::TEXT || ', "type": "W"}]')::jsonb
#  ) AS relways ON n.id = ANY(relways.nodes);
#"""
sql = """CREATE OR REPLACE FUNCTION convert_coord(c INTEGER) RETURNS FLOAT AS $$
BEGIN
  IF c IS NULL THEN
    RETURN NULL;
  END IF;
  RETURN c::FLOAT * 10.0^(-7);
END;
$$ LANGUAGE plpgsql STABLE LEAKPROOF PARALLEL SAFE STRICT;

CREATE TABLE landuse_nodes AS
WITH landuse_relation_nodes AS (
SELECT
    r.landuse_id AS landuse_id,
    r.feature AS feature,
    n.id AS node_id,
    ST_SetSRID(ST_MakePoint(convert_coord(n.lon), convert_coord(n.lat)), 4326) AS geom,
    n.created AS last_modified,
    n.version AS version
  FROM planet_osm_ways AS w
  JOIN (
    SELECT
        l.area_id AS landuse_id,
        l.feature AS feature,
        jsonb_path_query(r.members, '($[*] ? (@.type == "W")).ref') AS way_id
      FROM land AS l
      JOIN planet_osm_rels AS r ON -l.area_id = r.id AND l.area_id < 0
  ) AS r ON r.way_id::BIGINT = w.id
  JOIN planet_osm_nodes AS n ON n.id = ANY(w.nodes)
)
-- The pure way-nodes part (simple polygons) takes 12 s.
SELECT
    l.area_id AS landuse_id,
    l.feature AS feature,
    n.id AS node_id,
    ST_SetSRID(ST_MakePoint(convert_coord(n.lon), convert_coord(n.lat)), 4326) AS geom,
    n.created AS last_modified,
    n.version AS version
  FROM planet_osm_ways AS w
  -- Instead of joining with the land table, we could write a WHERE condition using the same implementation as the Lua code.
  JOIN land AS l ON l.area_id = w.id
  JOIN planet_osm_nodes AS n ON n.id = ANY(w.nodes)
UNION ALL
-- The relations-way-nodes part (multipolygons) takes 3 s.
SELECT * FROM landuse_relation_nodes;
"""
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX landuse_nodes_geom_idx ON landuse_nodes USING gist(geom);")
send_sql(conn, "CREATE INDEX landuse_nodes_area_id_idx ON landuse_nodes USING btree(landuse_id);")

Find median age of landuse per polygon:

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_polygon_age;")
sql = """CREATE TABLE landuse_polygon_age AS
SELECT
    n.landuse_id,
    n.feature,
    l.tags->'source' AS source,
    MIN(n.last_modified) AS oldest_timestamp,
    MAX(n.last_modified) AS newest_timestamp,
    to_timestamp((PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY EXTRACT(EPOCH FROM n.last_modified)))::BIGINT) AS median_timestamp,
    COUNT(1) AS node_count,
    l.geom AS geom
  FROM landuse_nodes AS n
  JOIN land AS l
    ON n.landuse_id = l.area_id
  GROUP BY n.landuse_id, n.feature, l.tags->'source', l.geom;
"""
send_sql(conn, sql)

Get node age per municipality:

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_node_age_per_municipality;")
sql = """CREATE TABLE landuse_node_age_per_municipality AS
SELECT
    MIN(n.last_modified) AS oldest_timestamp,
    MAX(n.last_modified) AS newest_timestamp,
    to_timestamp((PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY EXTRACT(EPOCH FROM n.last_modified)))::BIGINT) AS median_timestamp,
    COUNT(1) AS node_count,
    c.area_id AS city_id,
    c.name AS name,
    ST_Area(c.geom) AS area
  FROM cities AS c
  JOIN landuse_nodes AS n
    ON c.geom && n.geom AND ST_Intersects(c.geom, n.geom)
  GROUP BY c.area_id, c.name, c.geom;
"""
send_sql(conn, sql)